In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import numpy as np
import pandas as pd
import datetime
import pickle
import re



In [42]:
pd.options.display.max_rows = 1000

In [32]:
def list_dates(start,end):
    # create datetime object for the start and end dates
    start = datetime.datetime.strptime(start, '%Y-%m-%d')
    end = datetime.datetime.strptime(end, '%Y-%m-%d')
    # generates list of dates between start and end dates
    step = datetime.timedelta(days=1)
    dates = []
    while start <= end:
        dates.append(start.date())
        start += step
    # return the list of dates in string format
    return [str(date) for date in dates]

def date_part(data,f_mat ='%Y-%m-%d'):
    # creates a pandas dataframe of 
    data = pd.DataFrame(data,columns=['date'])
    date_time = data['date']
    fld = pd.to_datetime(date_time, format=f_mat)
    for n in ('Month', 'Day','Year'):
        data[n] = getattr(fld.dt,n.lower())
    data['Month'] = data['Month'].map(month_dict)
    return data

def scrapper(dates,zipcode):
    data=[] # list to append scrapped data
    # submits the zipcode to find the closest weather center
    search= driver.find_element_by_xpath('//*[@id="history-icao-search"]')
    search.clear()
    search.send_keys(zipcode)
    search.submit()
    time.sleep(3) # sleep timer to wait for page to load (not necessary)
    
    # iterates through provided list of dates to scrap weather for
    for i,v in dates.iterrows():
        # inputs month, day, year into website to view information
        month = driver.find_element_by_class_name('month')
        month.send_keys(v['Month'])
        day = driver.find_element_by_class_name('day')
        day.send_keys(v['Day'])
        year = driver.find_element_by_class_name('year')
        year.send_keys(v['Year'])
        year.submit() # submits to search for month, day, year
#         time.sleep(3) # sleep timer to wait for page to load (not necessary)

        # scraps table on the bottom for weather information
        links = driver.find_elements_by_id('observations_details') # locates the data
        x = links[0].text # scrapes that data
        x= re.sub(r'[^\x00-\x7F]+',' ', x) # removes unicode
        x = x.split('\n') # breaks the data into observations per row
        x = x[1:-1] # removes the last line
        data.extend([i+' '+v['date'] for i in x]) # appends all scraped data
    return data


def rreplace(df, col,letter, rletter):
    ''' this does some pre processing of the data before splitting'''
    return df[col].apply(lambda x: x.replace(letter,rletter))

month_dict ={
    1:'January',
    2:'February',
    3:'March',
    4:'April',
    5:'May',
    6:'June',
    7:'July',
    8:'August',
    9:'September',
    10:'October',
    11:'November',
    12:'December'
}

def weather_scrapper(start_date,end_date, zipcode):
    dates = list_dates(start_date,end_date)
    dates = date_part(dates,'%Y-%m-%d')
    data = scrapper(dates,zipcode)
    return preprocess_data(data)

def preprocess_data(data):
    dt = [i.replace('Calm Calm', 'Calm 0.0 mph') for i in data]
    dt = [i.replace(' AM', 'AM') for i in dt]
    dt = [i.replace(' PM', 'PM') for i in dt]
    dt = [i.replace('%', '') for i in dt]   
    dt = [i.replace(' mi', '') for i in dt]
    dt = [i.replace(' mph', '') for i in dt]
    dt = [i.replace(' in', '') for i in dt]
    dt = [i.replace('  ', ' ') for i in dt]
    dt = [i.replace('  ', ' ') for i in dt]
    dt = [i.replace('Fog', ' ',1) for i in dt]

    dt = [i.replace(' Clou', 'Clou') for i in dt]
    dt = [i.replace('t Rain', 'tRain') for i in dt]
    dt = [i.replace('y Rain', 'yRain') for i in dt]
    dt = [i.replace('Rain , Thunderstorm', 'RainThunderstorm') for i in dt]
    
    dt = [i.replace('Thunderstorm', '',1) for i in dt]
    dt = [i.replace('Light Thunderstorms and Rain', 'LightThunderstormsandRain') for i in dt]
    dt = [i.replace('Thunderstorms and Rain', 'ThunderstormsandRain') for i in dt]
    
    dt = [i.replace('Rain', '',1) for i in dt]
    dt = [i.replace('Light Drizzle', 'LightDrizzle') for i in dt]
    
    dt = [i.replace('F', '') for i in dt]
    dt = [i.replace(' og', ' Fog') for i in dt]
    dt = [i.replace('Patches of Fog', 'PatchesofFog',1) for i in dt]
  
    dt = [i.split() for i in dt]
    dt = [ i[:2] +i[-10:] for i in dt]
    
    dt = pd.DataFrame(dt,columns = ['time','temp(F)','dewpoint(F)','humidity(%)','pressure(in)','visibility(mi)','winddir','windspeed(mph)','gustspeed(mph)','precip(in)','conditions','date'])
    dt['time'] = [datetime.datetime.strftime(datetime.datetime.strptime(val, "%I:%M%p"), "%H:%M") for val in dt['time']]
    return dt

In [28]:
driver = webdriver.Chrome('/usr/local/bin/chromedriver')
x = 'https://www.wunderground.com/history/airport/KSFO/2018/2/24/DailyHistory.html?req_city=San%20Francisco&req_statename=California'
driver.get(x)

In [130]:
# YYYY-MM-DD
start = '2014-1-1'
end = '2014-1-10'
zipcode = '94121'

In [131]:
weather = weather_scrapper(start,end,zipcode)

In [6]:
weather.to_csv('weather2.csv',index=False)

In [133]:
weather = pd.read_csv('weather2.csv')

In [134]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35801 entries, 0 to 35800
Data columns (total 12 columns):
time              35801 non-null object
temp(F)           35801 non-null object
dewpoint(F)       35801 non-null object
humidity(%)       35781 non-null object
pressure(in)      35801 non-null object
visibility(mi)    35801 non-null object
winddir           35801 non-null object
windspeed(mph)    35801 non-null object
gustspeed(mph)    35794 non-null object
precip(in)        3093 non-null object
conditions        35801 non-null object
date              35801 non-null object
dtypes: object(12)
memory usage: 3.3+ MB


In [135]:
grouped = weather.groupby('date', as_index=False)

avg_name = ['date','avg_temp','avg_dewpoint','avg_humidity','avg_pressure','avg_visibility', 'avg_windspeed', 'avg_gustspeed','avg_precip']

max_name = ['date','max_temp','max_dewpoint',
 'max_humidity', 'max_pressure',
 'max_visibility', 'max_windspeed',
 'max_gustspeed', 'max_precip']

min_name = ['date',
 'min_temp',
 'min_dewpoint',
 'min_humidity',
 'min_pressure',
 'min_visibility',
 'min_windspeed',
 'min_gustspeed',
 'min_precip']

avg_group = grouped.aggregate(np.mean)
avg_group.columns = avg_name
avg_group

max_group = grouped.aggregate(np.max).drop(['time','winddir','conditions'],axis=1)
max_group.columns = max_name
max_group

min_group = grouped.aggregate(np.min).drop(['time','winddir','conditions'],axis=1)
min_group.columns = min_name
min_group

stat_group = avg_group.merge(max_group, on='date')
stat_group = stat_group.merge(min_group, on='date')

x = df.merge(stat_group, on='date')

x.shape

DataError: No numeric types to aggregate

In [147]:
grouped = df.groupby('date', as_index=False)

In [172]:
avg_name = ['date','avg_temp','avg_dewpoint','avg_humidity','avg_pressure','avg_visibility', 'avg_windspeed', 'avg_gustspeed','avg_precip']

max_name = ['date','max_temp','max_dewpoint',
 'max_humidity', 'max_pressure',
 'max_visibility', 'max_windspeed',
 'max_gustspeed', 'max_precip']

min_name = ['date',
 'min_temp',
 'min_dewpoint',
 'min_humidity',
 'min_pressure',
 'min_visibility',
 'min_windspeed',
 'min_gustspeed',
 'min_precip']

In [173]:
avg_group = grouped.aggregate(np.mean)
avg_group.columns = avg_name
avg_group

,date,avg_temp,avg_dewpoint,avg_humidity,avg_pressure,avg_visibility,avg_windspeed,avg_gustspeed,avg_precip
0,1/18/2014,53.166667,38.762500,59.500000,30.111667,8.916667,1.258333,0.000000,0.000000
1,1/19/2014,54.258333,38.370833,56.541667,30.096667,9.666667,1.933333,0.000000,0.000000
2,1/20/2014,53.320833,40.833333,63.333333,30.167917,9.416667,1.733333,0.000000,0.000000
3,1/21/2014,53.741667,36.812500,54.916667,30.211250,10.000000,1.929167,0.000000,0.000000
4,1/23/2014,56.487500,38.595833,55.416667,29.924583,9.208333,4.566667,0.000000,0.000000
5,1/24/2014,54.052000,41.724000,65.000000,30.160800,7.720000,2.456000,0.000000,0.000000
6,1/25/2014,56.379167,43.104167,62.541667,30.195000,9.166667,1.695833,0.000000,0.000000
7,1/26/2014,54.895833,43.445833,66.375000,30.090833,9.958333,5.816667,0.000000,0.000000
8,1/27/2014,53.244444,45.944444,76.444444,30.145556,9.814815,6.240741,0.000000,0.000000
9,1/28/2014,58.231034,52.608621,81.655172,30.221724,9.475862,5.582759,0.000000,0.000000


In [175]:
max_group = grouped.aggregate(np.max).drop(['time','winddir','conditions'],axis=1)
max_group.columns = max_name
max_group

,date,max_temp,max_dewpoint,max_humidity,max_pressure,max_visibility,max_windspeed,max_gustspeed,max_precip
0,1/18/2014,62.1,46.90,76,30.18,10.0,4.6,0.0,0.00
1,1/19/2014,66.0,44.10,74,30.14,10.0,4.6,0.0,0.00
2,1/20/2014,63.0,48.00,76,30.23,10.0,8.1,0.0,0.00
3,1/21/2014,66.9,44.10,77,30.27,10.0,10.4,0.0,0.00
4,1/23/2014,68.0,46.90,83,30.02,10.0,15.0,0.0,0.00
5,1/24/2014,63.0,46.90,77,30.28,10.0,9.2,0.0,0.00
6,1/25/2014,64.0,48.00,83,30.27,10.0,8.1,0.0,0.00
7,1/26/2014,64.9,48.00,83,30.14,10.0,17.3,0.0,0.00
8,1/27/2014,59.0,48.00,83,30.21,10.0,18.4,0.0,0.00
9,1/28/2014,65.3,57.20,88,30.27,10.0,16.1,0.0,0.00


In [176]:
min_group = grouped.aggregate(np.min).drop(['time','winddir','conditions'],axis=1)
min_group.columns = min_name
min_group

,date,min_temp,min_dewpoint,min_humidity,min_pressure,min_visibility,min_windspeed,min_gustspeed,min_precip
0,1/18/2014,45.0,32.0,38,30.06,7.0,0.0,0.0,0.0
1,1/19/2014,44.1,34.0,32,30.06,8.0,0.0,0.0,0.0
2,1/20/2014,45.0,34.0,43,30.12,7.0,0.0,0.0,0.0
3,1/21/2014,44.1,25.0,21,30.16,10.0,0.0,0.0,0.0
4,1/23/2014,45.0,27.0,21,29.88,7.0,0.0,0.0,0.0
5,1/24/2014,46.9,32.0,31,30.03,4.0,0.0,0.0,0.0
6,1/25/2014,50.0,34.0,37,30.13,7.0,0.0,0.0,0.0
7,1/26/2014,48.9,33.1,45,30.03,9.0,0.0,0.0,0.0
8,1/27/2014,48.9,43.0,64,30.08,8.0,0.0,0.0,0.0
9,1/28/2014,53.1,46.9,74,30.20,1.8,0.0,0.0,0.0


In [180]:
stat_group = avg_group.merge(max_group, on='date')
stat_group = stat_group.merge(min_group, on='date')

In [181]:
stat_group

,date,avg_temp,avg_dewpoint,avg_humidity,avg_pressure,avg_visibility,avg_windspeed,avg_gustspeed,avg_precip,max_temp,max_dewpoint,max_humidity,max_pressure,max_visibility,max_windspeed,max_gustspeed,max_precip,min_temp,min_dewpoint,min_humidity,min_pressure,min_visibility,min_windspeed,min_gustspeed,min_precip
0,1/18/2014,53.166667,38.762500,59.500000,30.111667,8.916667,1.258333,0.000000,0.000000,62.1,46.90,76,30.18,10.0,4.6,0.0,0.00,45.0,32.0,38,30.06,7.0,0.0,0.0,0.0
1,1/19/2014,54.258333,38.370833,56.541667,30.096667,9.666667,1.933333,0.000000,0.000000,66.0,44.10,74,30.14,10.0,4.6,0.0,0.00,44.1,34.0,32,30.06,8.0,0.0,0.0,0.0
2,1/20/2014,53.320833,40.833333,63.333333,30.167917,9.416667,1.733333,0.000000,0.000000,63.0,48.00,76,30.23,10.0,8.1,0.0,0.00,45.0,34.0,43,30.12,7.0,0.0,0.0,0.0
3,1/21/2014,53.741667,36.812500,54.916667,30.211250,10.000000,1.929167,0.000000,0.000000,66.9,44.10,77,30.27,10.0,10.4,0.0,0.00,44.1,25.0,21,30.16,10.0,0.0,0.0,0.0
4,1/23/2014,56.487500,38.595833,55.416667,29.924583,9.208333,4.566667,0.000000,0.000000,68.0,46.90,83,30.02,10.0,15.0,0.0,0.00,45.0,27.0,21,29.88,7.0,0.0,0.0,0.0
5,1/24/2014,54.052000,41.724000,65.000000,30.160800,7.720000,2.456000,0.000000,0.000000,63.0,46.90,77,30.28,10.0,9.2,0.0,0.00,46.9,32.0,31,30.03,4.0,0.0,0.0,0.0
6,1/25/2014,56.379167,43.104167,62.541667,30.195000,9.166667,1.695833,0.000000,0.000000,64.0,48.00,83,30.27,10.0,8.1,0.0,0.00,50.0,34.0,37,30.13,7.0,0.0,0.0,0.0
7,1/26/2014,54.895833,43.445833,66.375000,30.090833,9.958333,5.816667,0.000000,0.000000,64.9,48.00,83,30.14,10.0,17.3,0.0,0.00,48.9,33.1,45,30.03,9.0,0.0,0.0,0.0
8,1/27/2014,53.244444,45.944444,76.444444,30.145556,9.814815,6.240741,0.000000,0.000000,59.0,48.00,83,30.21,10.0,18.4,0.0,0.00,48.9,43.0,64,30.08,8.0,0.0,0.0,0.0
9,1/28/2014,58.231034,52.608621,81.655172,30.221724,9.475862,5.582759,0.000000,0.000000,65.3,57.20,88,30.27,10.0,16.1,0.0,0.00,53.1,46.9,74,30.20,1.8,0.0,0.0,0.0


In [183]:
x = df.merge(stat_group, on='date')

In [184]:
x.shape

(2253, 36)

In [185]:
df.shape

(2253, 12)